# --- Importacion de Librerias ---

In [13]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F 
from datetime import datetime
import logging
import sys
import locale
reload(sys)
sys.setdefaultencoding('utf-8')
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

# --- Lectura de datos ---

In [ ]:
if __name__ == '__main__':
    spark = SparkSession \
        .builder \
        .appName('IPC_2002-2024_085') \
        .getOrCreate()
    sc = spark.sparkContext


In [10]:
    ruta_hdfs = "hdfs:///user/xuedua085/datasetipc/IPC2002-2024.json"
    df = spark.read.option("mode", "FAILFAST").json(ruta_hdfs)

In [11]:
    print("## Esquema del DataFrame:")
    df.printSchema()

    print("\n## Algunos datos de ejemplo (primeras 5 filas):")
    df.show(5, truncate=False)

## Esquema del DataFrame:
root
 |-- Grupos ECOICOP: string (nullable = true)
 |-- Periodo: long (nullable = true)
 |-- Provincias: string (nullable = true)
 |-- Tipo de dato: string (nullable = true)
 |-- Total: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)


## Algunos datos de ejemplo (primeras 5 filas):
+--------------+-------+--------------+------------+-------+----------+----------+
|Grupos ECOICOP|Periodo|Provincias    |Tipo de dato|Total  |lat       |lon       |
+--------------+-------+--------------+------------+-------+----------+----------+
|Índice general|2024   |01 Araba/Álava|Media anual |114.19 |42.8440722|-2.6820829|
|Índice general|2023   |01 Araba/Álava|Media anual |111.378|42.8440722|-2.6820829|
|Índice general|2022   |01 Araba/Álava|Media anual |108.092|42.8440722|-2.6820829|
|Índice general|2021   |01 Araba/Álava|Media anual |100.0  |42.8440722|-2.6820829|
|Índice general|2020   |01 Araba/Álava|Media anual |96.923 |42

# --- Consultas Interesantes ---

In [14]:
    # 1. Evolución del IPC "Índice general" (Media anual) para "15 Coruña, A" - 5 periodos más recientes.
    print("\n## 1. Evolución del IPC 'Índice general' (Media anual) para '15 Coruña, A' (5 más recientes):")
    evolucion_ipc_coru = df.filter(
        (F.col("Provincias") == "15 Coruña, A") &
        (F.col("Grupos ECOICOP") == "Índice general") &
        (F.col("Tipo de dato") == "Media anual")
    ).select("Periodo", "Total").orderBy(F.col("Periodo").desc()) # Ordenamos descendente por periodo

    evolucion_ipc_coru.show(5)



## 1. Evolución del IPC 'Índice general' (Media anual) para '15 Coruña, A' (5 más recientes):
+-------+-------+
|Periodo|  Total|
+-------+-------+
|   2024|115.693|
|   2023|112.362|
|   2022|108.481|
|   2021|  100.0|
|   2020| 96.755|
+-------+-------+
only showing top 5 rows



In [15]:
    # 2. Top 5 "Grupos ECOICOP" con el mayor valor "Total" (Media anual) en el año 2023 para "15 Coruña, A".
    #    Excluiremos el "Índice general" para ver los grupos específicos.
    print("\n## 2. Top 5 'Grupos ECOICOP' por IPC (Media anual) en 2023 para '15 Coruña, A' (excl. Índice general):")
    top_grupos_2023_coru = df.filter(
        (F.col("Provincias") == "15 Coruña, A") &
        (F.col("Periodo") == 2023) &
        (F.col("Tipo de dato") == "Media anual") &
        (F.col("Grupos ECOICOP") != "Índice general")
    ).select("Grupos ECOICOP", "Total").orderBy(F.col("Total").desc())

    top_grupos_2023_coru.show(5, truncate=False)



## 2. Top 5 'Grupos ECOICOP' por IPC (Media anual) en 2023 para '15 Coruña, A' (excl. Índice general):
+-------------------------------------------------------------------------------------+-------+
|Grupos ECOICOP                                                                       |Total  |
+-------------------------------------------------------------------------------------+-------+
|01 Alimentos y bebidas no alcohólicas                                                |123.1  |
|07 Transporte                                                                        |113.195|
|11 Restaurantes y hoteles                                                            |112.589|
|02 Bebidas alcohólicas y tabaco                                                      |112.583|
|05 Muebles, artículos del hogar y artículos para el mantenimiento corriente del hogar|112.522|
+-------------------------------------------------------------------------------------+-------+
only showing top 5 rows



In [19]:
    # 3. Años con la mayor y menor "Variación de las medias anuales" para el "Índice general" en "15 Coruña, A".
    print("\n## 3. Años con mayor y menor 'Variación de las medias anuales' del 'Índice general' en '15 Coruña, A':")

    variacion_general_coru = df.filter(
        (F.col("Provincias") == "15 Coruña, A") &
        (F.col("Grupos ECOICOP") == "Índice general") &
        (F.col("Tipo de dato") == "Variación de las medias anuales") &
        F.col("Total").isNotNull()
    )
    
    # Año con mayor variación
    mayor_variacion_df = variacion_general_coru.orderBy(F.col("Total").desc()).limit(1)
    print("\n### Dato del Año con MAYOR variación:")
    if not mayor_variacion_df.rdd.isEmpty():
        fila_mayor = mayor_variacion_df.first()
        print("  Periodo:", fila_mayor['Periodo'], ", Variación (Total):", fila_mayor['Total'])
    else:
        print("  No se encontraron datos para la mayor variación.")

    # Año con menor variación
    menor_variacion_df = variacion_general_coru.orderBy(F.col("Total").asc()).limit(1)
    print("\n### Dato del Año con MENOR variación:")
    if not menor_variacion_df.rdd.isEmpty():
        fila_menor = menor_variacion_df.first()
        print("  Periodo:", fila_menor['Periodo'], ", Variación (Total):", fila_menor['Total'])
    else:
        print("  No se encontraron datos para la menor variación.")


## 3. Años con mayor y menor 'Variación de las medias anuales' del 'Índice general' en '15 Coruña, A':

### Dato del Año con MAYOR variación:
('  Periodo:', 2022, ', Variaci\xc3\xb3n (Total):', 8.5)

### Dato del Año con MENOR variación:
('  Periodo:', 2015, ', Variaci\xc3\xb3n (Total):', -0.6)


In [24]:
    # 4. Comparar el IPC "Media anual" del "Índice general" de "15 Coruña, A"
    #    entre el primer año disponible (p.ej. 2002) y el último (p.ej. 2024).
    print("\n## 4. Comparación IPC 'Índice general' (Media anual) en '15 Coruña, A' entre 2002 y 2024:")

    # Obtenemos el valor para 2002
    ipc_2002_coru_df = df.filter(
        (F.col("Provincias") == "15 Coruña, A") &
        (F.col("Grupos ECOICOP") == "Índice general") &
        (F.col("Tipo de dato") == "Media anual") &
        (F.col("Periodo") == 2002)
    ).select("Total")

    total_2002 = None
    if not ipc_2002_coru_df.rdd.isEmpty():
        total_2002 = ipc_2002_coru_df.first()["Total"]
        print("  IPC en 2002: ", total_2002)
    else:
        print("  No se encontraron datos del IPC para 2002.")

    # Obtenemos el valor para 2024
    ipc_2024_coru_df = df.filter(
        (F.col("Provincias") == "15 Coruña, A") &
        (F.col("Grupos ECOICOP") == "Índice general") &
        (F.col("Tipo de dato") == "Media anual") &
        (F.col("Periodo") == 2024)
    ).select("Total")

    total_2024 = None
    if not ipc_2024_coru_df.rdd.isEmpty():
        total_2024 = ipc_2024_coru_df.first()["Total"]
        print("  IPC en 2024: ",total_2024)
    else:
        print("  No se encontraron datos del IPC para 2024.")

    # Calcular e imprimir el incremento porcentual si ambos valores existen
    if total_2002 is not None and total_2024 is not None:
        if total_2002 != 0: # Evitar división por cero
            incremento_porcentual = ((total_2024 - total_2002) / total_2002) * 100
            print("  Incremento porcentual (2002-2024): {:.2f}%".format(incremento_porcentual))
        else:
            print("  No se puede calcular el incremento porcentual porque el valor de 2002 es cero.")


## 5. Comparación IPC 'Índice general' (Media anual) en '15 Coruña, A' entre 2002 y 2024:
('  IPC en 2002: ', 70.412)
('  IPC en 2024: ', 115.693)
  Incremento porcentual (2002-2024): 64.31%


In [ ]:
if '__main__' == __name__ and 'spark' in locals(): # Solo si 'spark' fue definido en este script
    spark.stop()
    print("SparkSession detenida.")